In [34]:
from keras.datasets import cifar10
from keras.models import *
from keras.layers import *
import numpy as np
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import *
from keras.preprocessing import image
from keras import regularizers,optimizers
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch < 2:
        lrate = 0.005
    if epoch > 20:
        lrate = 0.0005
    if epoch > 30:
        lrate = 0.0001
    if epoch > 42:
        lrate = 0.00007
    if epoch > 60:
        lrate = 0.00003
    if epoch > 100:
        lrate = 0.00001
    if epoch > 120:
        lrate = 0.0001
    return lrate
 

In [35]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [36]:
x_train =np.asarray([one/255 for one in x_train])
y_train = to_categorical(y_train)

x_test = np.asarray([one/255 for one in x_test])
y_test_pure = y_test
y_test =to_categorical(y_test)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [37]:
def conv2d_bn(x,
              filters,
              kernel_size,
              strides=1,
              padding='same',
              activation='relu',
              use_bias=True,
              name=None):
    
    x = layers.Conv2D(filters,
                      kernel_size,
                      strides=strides,
                      padding=padding,
                      use_bias=use_bias,
                      name=name)(x)
    if activation is not None:
        ac_name = None if name is None else name + '_ac'
        x = layers.Activation(activation, name=ac_name)(x)
    return x

In [38]:
"""

my inception
img_input = Input(shape=(32,32,3))
layer1 =  Conv2D(16,(3,3),padding='same',use_bias=True)(img_input)
layer1 = Conv2D(32, (3, 3),padding='same',use_bias=True)(layer1)
layer1 = MaxPooling2D((2, 2))(layer1)


branch_1 = Conv2D(8,(1,1),padding='same',use_bias=True)(layer1)

branch_2 = Conv2D(16,(2,2),padding='same',use_bias=True)(layer1)
branch_2 = Conv2D(8,(2,2),padding='same',use_bias=True)(layer1)
branch_2 = MaxPool2D(1,1)(branch_2)

branch_3 = Conv2D(16,(1,1),padding='same',use_bias=True)(layer1)
branch_3 = Conv2D(32,(3,3),padding='same',use_bias=True)(branch_3)
branch_3 = Conv2D(8,(2,2),padding='same',use_bias=True)(branch_3)

branch_pool =AveragePooling2D((3, 3),strides=(1, 1),padding='same')(layer1)
branch_pool = Conv2D(8,(1,1),padding='same',use_bias=True)(branch_pool)

concat_conv = concatenate([branch_1,branch_2,branch_3,branch_pool],axis=1)

branch_1 = Conv2D(8,(1,1),padding='same',use_bias=True)(layer1)

branch_2 = Conv2D(32,(2,2),padding='same',use_bias=True)(layer1)
branch_2 = Conv2D(16,(2,2),padding='same',use_bias=True)(layer1)
branch_2 = MaxPool2D(1,1)(branch_2)

branch_3 = Conv2D(64,(1,1),padding='same',use_bias=True)(layer1)
branch_3 = Conv2D(96,(3,3),padding='same',use_bias=True)(branch_3)
branch_3 = Conv2D(16,(2,2),padding='same',use_bias=True)(branch_3)

branch_pool =AveragePooling2D((3, 3),strides=(1, 1),padding='same')(layer1)
branch_pool = Conv2D(16,(1,1),padding='same',use_bias=True)(branch_pool)

concat_conv = concatenate([branch_1,branch_2,branch_3,branch_pool],axis=3)
flt = GlobalAveragePooling2D()(concat_conv)
Den = Dense(256,activation='relu')(flt)
Den = Dropout(0.5)(Den)
res = Dense(100,activation='softmax')(Den)

model = Model(img_input,res)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['acc'])
model.summary()
"""

"\n\nmy inception\nimg_input = Input(shape=(32,32,3))\nlayer1 =  Conv2D(16,(3,3),padding='same',use_bias=True)(img_input)\nlayer1 = Conv2D(32, (3, 3),padding='same',use_bias=True)(layer1)\nlayer1 = MaxPooling2D((2, 2))(layer1)\n\n\nbranch_1 = Conv2D(8,(1,1),padding='same',use_bias=True)(layer1)\n\nbranch_2 = Conv2D(16,(2,2),padding='same',use_bias=True)(layer1)\nbranch_2 = Conv2D(8,(2,2),padding='same',use_bias=True)(layer1)\nbranch_2 = MaxPool2D(1,1)(branch_2)\n\nbranch_3 = Conv2D(16,(1,1),padding='same',use_bias=True)(layer1)\nbranch_3 = Conv2D(32,(3,3),padding='same',use_bias=True)(branch_3)\nbranch_3 = Conv2D(8,(2,2),padding='same',use_bias=True)(branch_3)\n\nbranch_pool =AveragePooling2D((3, 3),strides=(1, 1),padding='same')(layer1)\nbranch_pool = Conv2D(8,(1,1),padding='same',use_bias=True)(branch_pool)\n\nconcat_conv = concatenate([branch_1,branch_2,branch_3,branch_pool],axis=1)\n\nbranch_1 = Conv2D(8,(1,1),padding='same',use_bias=True)(layer1)\n\nbranch_2 = Conv2D(32,(2,2),padd

In [39]:
def make_conv_model_list():
    weight_decay = 1e-4
    model = Sequential()
    model.add(Conv2D(32,(2,2),use_bias=True,activation='elu',padding='same',input_shape=(32,32,3),kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(2,2))
    model.add(BatchNormalization())
    model.add(ZeroPadding2D(padding=(1, 1), data_format=None))
    model.add(Conv2D(64,(4,4),use_bias=True,padding='same',activation='elu',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(2,2))
    model.add(BatchNormalization())
    model.add(ZeroPadding2D(padding=(1, 1), data_format=None))
    model.add(Conv2D(128,(2,2),use_bias=True,padding='same',activation='elu',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(2,2))
    model.add(Flatten())
    model.add(Dense(64,activation='elu'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(32,activation='elu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    opt_rms = optimizers.rmsprop(lr=0.001,decay=1e-6)
    model.compile(optimizer=opt_rms,loss='categorical_crossentropy',metrics=['acc'])
    model.summary()
    return model

In [ ]:
ensemble_model_list = []
ensemble_size=5
ensemble_call_back =[LearningRateScheduler(lr_schedule)]
for i in range(ensemble_size):
    this_model = make_conv_model_list()
    print('i_th model training')
    this_model.fit_generator(datagen.flow(x_train, y_train, batch_size=125),\
                    steps_per_epoch=x_train.shape[0] // 125,epochs=200,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=ensemble_call_back)
    
    ensemble_model_list.append(this_model)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 32, 32, 32)        416       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
zero_padding2d_19 (ZeroPaddi (None, 18, 18, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 18, 18, 64)        32832     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 9, 9, 64)          0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 9, 9, 64)          256       
__________

400/400 [==============================] - 22s 55ms/step - loss: 0.8461 - acc: 0.7455 - val_loss: 0.7248 - val_acc: 0.7732
Epoch 43/125
400/400 [==============================] - 22s 55ms/step - loss: 0.8422 - acc: 0.7464 - val_loss: 0.7223 - val_acc: 0.7750
Epoch 44/125
400/400 [==============================] - 21s 53ms/step - loss: 0.8410 - acc: 0.7445 - val_loss: 0.7549 - val_acc: 0.7640
Epoch 45/125
400/400 [==============================] - 21s 53ms/step - loss: 0.8358 - acc: 0.7472 - val_loss: 0.7278 - val_acc: 0.7718
Epoch 46/125
400/400 [==============================] - 22s 54ms/step - loss: 0.8358 - acc: 0.7488 - val_loss: 0.7185 - val_acc: 0.7753
Epoch 47/125
400/400 [==============================] - 22s 54ms/step - loss: 0.8330 - acc: 0.7486 - val_loss: 0.7221 - val_acc: 0.7725
Epoch 48/125
400/400 [==============================] - 21s 53ms/step - loss: 0.8302 - acc: 0.7485 - val_loss: 0.7392 - val_acc: 0.7697
Epoch 49/125
400/400 [==============================] - 21s 5

400/400 [==============================] - 22s 54ms/step - loss: 0.7952 - acc: 0.7588 - val_loss: 0.7059 - val_acc: 0.7799
Epoch 103/125
400/400 [==============================] - 22s 54ms/step - loss: 0.8002 - acc: 0.7596 - val_loss: 0.7016 - val_acc: 0.7816
Epoch 104/125
400/400 [==============================] - 22s 54ms/step - loss: 0.8026 - acc: 0.7574 - val_loss: 0.6972 - val_acc: 0.7832
Epoch 105/125
400/400 [==============================] - 22s 55ms/step - loss: 0.8019 - acc: 0.7589 - val_loss: 0.7048 - val_acc: 0.7804
Epoch 106/125
400/400 [==============================] - 22s 55ms/step - loss: 0.7990 - acc: 0.7596 - val_loss: 0.6977 - val_acc: 0.7835
Epoch 107/125
400/400 [==============================] - 22s 55ms/step - loss: 0.7945 - acc: 0.7618 - val_loss: 0.7055 - val_acc: 0.7801
Epoch 108/125
400/400 [==============================] - 21s 53ms/step - loss: 0.8002 - acc: 0.7588 - val_loss: 0.7054 - val_acc: 0.7816
Epoch 109/125
400/400 [==============================] 

In [ ]:
all_predicted = np.zeros(y_test.shape)

In [ ]:
for one in ensemble_model_list:
    all_predicted =all_predicted+one.predict(x_test)

In [ ]:
final_res = []
for one in all_predicted:
    final_res.append(np.argmax(one))

In [ ]:
final_res = np.asarray(final_res)

In [ ]:
entire = y_test_pure.shape[0]

In [ ]:
ele = 0
for one in range(entire):
    if(y_test_pure[one]==final_res[one]):
        ele=ele+1
    
res_acc = ele/entire

In [ ]:
res_acc